roun out


In [6]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor  # Importe o XGBoost
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Carregar os dados de treinamento
train_data_path = 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/bc_train.csv'
train_df = pd.read_csv(train_data_path)

# Normalizar os dados de treinamento
scaler = StandardScaler()
cols_to_scale = train_df.columns.difference(['Date', 'Preco_Real'])
train_df[cols_to_scale] = scaler.fit_transform(train_df[cols_to_scale])

# Preparar os dados de treinamento
y_train = train_df['Preco_Real']
X_train = train_df.drop(['Date', 'Preco_Real'], axis=1)

# Carregar os dados de teste
test_data_path = 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/bc_test.csv'
test_df = pd.read_csv(test_data_path)

# Normalizar os dados de teste usando o mesmo scaler dos dados de treinamento
test_df[cols_to_scale] = scaler.transform(test_df[cols_to_scale])

# Preparar os dados de teste
y_test = test_df['Preco_Real']
X_test = test_df.drop(['Date', 'Preco_Real'], axis=1)

# Definir e treinar os modelos
models = {
    "Linear Regression": LinearRegression(),
    "Lasso Regression": Lasso(),
    "Random Forest": RandomForestRegressor(),
    "XGBoost": XGBRegressor()  # Adicione o XGBoost
}

# Função para avaliar o modelo com validação hold-out e retornar previsões
def evaluate_and_predict(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    r2 = r2_score(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    return predictions, r2, mae, mse

# Preparando o DataFrame para comparação
comparison_df = test_df[['Date', 'Preco_Real']].copy()
comparison_df = comparison_df.reset_index(drop=True)

# Avaliando cada modelo, armazenando previsões e métricas
for name, model in models.items():
    predictions, r2, mae, mse = evaluate_and_predict(model, X_train, y_train, X_test, y_test)
    comparison_df[f'Predicted_{name}'] = predictions
    print(f'{name}: R2={r2}, MAE={mae}, MSE={mse}')

# Função para calcular a diferença diária nas previsões e nos valores reais
def calculate_daily_difference(df, column_name, days=1):
    df[f'Differenca_{days}_Dias_{column_name}'] = df[column_name].diff(periods=days).fillna(0)
    return df

days = 1  # Define o número de dias para calcular a diferença
comparison_df = calculate_daily_difference(comparison_df, 'Preco_Real', days)
for model_name in models.keys():
    comparison_df = calculate_daily_difference(comparison_df, f'Predicted_{model_name}', days)

# Exibindo as diferenças calculadas
comparison_df[['Date', f'Differenca_{days}_Dias_Preco_Real'] + 
              [f'Differenca_{days}_Dias_Predicted_{model_name}' for model_name in models.keys()]].head()


Linear Regression: R2=-1.0438212835862961, MAE=164.37807917470542, MSE=33888.1372953494
Ridge Regression: R2=-1.1768537008781728, MAE=169.55472132607053, MSE=36093.91764323224
Lasso Regression: R2=-1.1259739881860726, MAE=166.2856224058094, MSE=35250.29266334538
Random Forest: R2=0.7820482652172407, MAE=50.96147156862751, MSE=3613.808296935601
XGBoost: R2=0.8333643274130891, MAE=46.04197075338924, MSE=2762.9483048632196


,Date,Differenca_1_Dias_Preco_Real,Differenca_1_Dias_Predicted_Linear Regression,Differenca_1_Dias_Predicted_Ridge Regression,Differenca_1_Dias_Predicted_Lasso Regression,Differenca_1_Dias_Predicted_Random Forest,Differenca_1_Dias_Predicted_XGBoost
0,2023-01-06,0.00,0.000000,0.000000,0.000000,0.0000,0.000000
1,2023-01-09,-20.62,26.625388,10.910766,-3.784665,-39.1529,-37.372009
2,2023-01-10,-34.48,-30.764866,-32.247534,-22.587064,-14.2304,-9.452820
3,2023-01-11,-23.81,-9.515803,-1.083551,-1.035089,-5.2330,-7.784851
4,2023-01-12,8.44,-3.899730,-2.874139,-2.673554,0.7532,-15.495972


janela deslisante 1

In [8]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
# Carregar os dados de treinamento
train_data_path = 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/bc_train.csv'
train_df = pd.read_csv(train_data_path)

# Normalizar os dados de treinamento
scaler = StandardScaler()
cols_to_scale = train_df.columns.difference(['Date', 'Preco_Real'])
train_df[cols_to_scale] = scaler.fit_transform(train_df[cols_to_scale])

# Preparar os dados de treinamento
y_train = train_df['Preco_Real']
X_train = train_df.drop(['Date', 'Preco_Real'], axis=1)

# Carregar os dados de teste
test_data_path = 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/bc_test.csv'
test_df = pd.read_csv(test_data_path)

# Normalizar os dados de teste usando o mesmo scaler dos dados de treinamento
test_df[cols_to_scale] = scaler.transform(test_df[cols_to_scale])

# Preparar os dados de teste
y_test = test_df['Preco_Real']
X_test = test_df.drop(['Date', 'Preco_Real'], axis=1)

# Definir os modelos
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "Random Forest": RandomForestRegressor(),
    "XGBoost": XGBRegressor()
}

# Inicializando DataFrame para armazenar resultados
results_df = pd.DataFrame(columns=['Date', 'Real_Value'] + [f'Predicted_{m}' for m in models.keys()])

# Definir o tamanho da janela deslizante
window_size = 1  # Por exemplo, 1 dia

# Inicializando DataFrame para armazenar resultados
results_df = pd.DataFrame(columns=['Date', 'Real_Value'] + [f'Predicted_{m}' for m in models.keys()])

# Usando tqdm para barra de progresso
for i in tqdm(range(window_size, len(X_test)), desc='Processing'):
    # Atualizando conjuntos de treino e teste
    X_train_updated = pd.concat([X_train, X_test.iloc[(i-window_size):i]])
    y_train_updated = pd.concat([y_train, y_test.iloc[(i-window_size):i]])
    # Armazenando a data e o valor real
    current_row = {'Date': test_df.iloc[i]['Date'], 'Real_Value': y_test.iloc[i]}
    
    # Avaliando cada modelo
    for name, model in models.items():
        # Treinando o modelo e fazendo a previsão
        model.fit(X_train_updated, y_train_updated)
        prediction = model.predict(X_test.iloc[[i]])

        # Armazenando a previsão
        current_row[f'Predicted_{name}'] = prediction[0]
    
    # Adicionando ao DataFrame de resultados
    results_df = results_df.append(current_row, ignore_index=True)

# Exibindo os resultados
print(results_df.head())


Processing:   0%|          | 0/203 [00:00<?, ?it/s]C:\Users\milen\AppData\Local\Temp\ipykernel_16240\578003378.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(current_row, ignore_index=True)
Processing:   0%|          | 1/203 [00:14<49:34, 14.72s/it]C:\Users\milen\AppData\Local\Temp\ipykernel_16240\578003378.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(current_row, ignore_index=True)
Processing:   1%|          | 2/203 [00:29<49:28, 14.77s/it]C:\Users\milen\AppData\Local\Temp\ipykernel_16240\578003378.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(current_row, ignore_index=True)
Processing:   1%|▏         | 3/203 [

         Date  Real_Value  Predicted_Linear Regression  \
0  2023-01-09     1002.85                  1152.910915   
1  2023-01-10      968.37                  1122.110790   
2  2023-01-11      944.56                  1112.436739   
3  2023-01-12      953.00                  1108.241660   
4  2023-01-13      945.34                  1085.672641   

   Predicted_Ridge Regression  Predicted_Lasso Regression  \
0                 1154.698832                 1151.058559   
1                 1122.666870                 1128.785936   
2                 1121.363439                 1127.442865   
3                 1117.927623                 1124.411258   
4                 1091.405998                 1097.086370   

   Predicted_Random Forest  Predicted_XGBoost  
0                 981.1373         991.156311  
1                 973.1688         975.654114  
2                 958.0182         958.982239  
3                 958.2699         957.520874  
4                 956.5310         960.09515

In [9]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
# Carregar os dados de treinamento
train_data_path = 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/bc_train.csv'
train_df = pd.read_csv(train_data_path)

# Normalizar os dados de treinamento
scaler = StandardScaler()
cols_to_scale = train_df.columns.difference(['Date', 'Preco_Real'])
train_df[cols_to_scale] = scaler.fit_transform(train_df[cols_to_scale])

# Preparar os dados de treinamento
y_train = train_df['Preco_Real']
X_train = train_df.drop(['Date', 'Preco_Real'], axis=1)

# Carregar os dados de teste
test_data_path = 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/bc_test.csv'
test_df = pd.read_csv(test_data_path)

# Normalizar os dados de teste usando o mesmo scaler dos dados de treinamento
test_df[cols_to_scale] = scaler.transform(test_df[cols_to_scale])

# Preparar os dados de teste
y_test = test_df['Preco_Real']
X_test = test_df.drop(['Date', 'Preco_Real'], axis=1)

# Definir os modelos
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "Random Forest": RandomForestRegressor(),
    "XGBoost": XGBRegressor()
}

# Inicializando DataFrame para armazenar resultados
results_df = pd.DataFrame(columns=['Date', 'Real_Value'] + [f'Predicted_{m}' for m in models.keys()])

# Definir o tamanho da janela deslizante
window_size = 30  # Por exemplo, 1 dia

# Inicializando DataFrame para armazenar resultados
results_df = pd.DataFrame(columns=['Date', 'Real_Value'] + [f'Predicted_{m}' for m in models.keys()])

# Usando tqdm para barra de progresso
for i in tqdm(range(window_size, len(X_test)), desc='Processing'):
    # Atualizando conjuntos de treino e teste
    X_train_updated = pd.concat([X_train, X_test.iloc[(i-window_size):i]])
    y_train_updated = pd.concat([y_train, y_test.iloc[(i-window_size):i]])
    # Armazenando a data e o valor real
    current_row = {'Date': test_df.iloc[i]['Date'], 'Real_Value': y_test.iloc[i]}
    
    # Avaliando cada modelo
    for name, model in models.items():
        # Treinando o modelo e fazendo a previsão
        model.fit(X_train_updated, y_train_updated)
        prediction = model.predict(X_test.iloc[[i]])

        # Armazenando a previsão
        current_row[f'Predicted_{name}'] = prediction[0]
    
    # Adicionando ao DataFrame de resultados
    results_df = results_df.append(current_row, ignore_index=True)

# Exibindo os resultados
print(results_df.head())


Processing:   0%|          | 0/174 [00:00<?, ?it/s]C:\Users\milen\AppData\Local\Temp\ipykernel_16240\3092982244.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(current_row, ignore_index=True)
Processing:   1%|          | 1/174 [00:22<1:03:49, 22.14s/it]C:\Users\milen\AppData\Local\Temp\ipykernel_16240\3092982244.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(current_row, ignore_index=True)
Processing:   1%|          | 2/174 [00:43<1:02:42, 21.87s/it]C:\Users\milen\AppData\Local\Temp\ipykernel_16240\3092982244.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(current_row, ignore_index=True)
Processing:   2%|▏         | 

         Date  Real_Value  Predicted_Linear Regression  \
0  2023-02-17     1157.53                  1208.823690   
1  2023-02-22     1179.20                  1169.745752   
2  2023-02-23     1150.14                  1164.477747   
3  2023-02-24     1141.39                  1156.808180   
4  2023-02-27     1141.47                  1165.491716   

   Predicted_Ridge Regression  Predicted_Lasso Regression  \
0                 1212.495195                 1201.460267   
1                 1161.875648                 1167.666204   
2                 1164.371061                 1164.039995   
3                 1157.417324                 1153.789958   
4                 1169.327120                 1169.359707   

   Predicted_Random Forest  Predicted_XGBoost  
0                1257.4245        1197.838623  
1                1261.5762        1218.776733  
2                1198.1525        1177.772339  
3                1135.5135        1177.993530  
4                1137.1450        1139.64892